# Python visualization with Python QT and VTK

date = Oktober 18, 2020 <br>
author = c.magg <br>

In [1]:
import os
import numpy as np
import cv2
import sys
import matplotlib.pyplot as plt
import logging

In [2]:
logging.basicConfig(level=logging.INFO)

In [3]:
sys.path.append(os.path.abspath('../../'))
sys.path.append(os.path.abspath('../utils'))

In [4]:
import vtk
from vtk.qt.QVTKRenderWindowInteractor import QVTKRenderWindowInteractor

In [5]:
import sys
from PyQt5.QtWidgets import QApplication, QLabel, QMainWindow, QToolBar 
from PyQt5.QtWidgets import QStatusBar, QCheckBox, QAction
from PyQt5.QtWidgets import QFrame, QVBoxLayout, QWidget
from PyQt5.QtWidgets import QPushButton, QFileDialog, QSlider
from PyQt5.QtCore import Qt

In [6]:
from SliceText import SliceText
from VTKSegmentationMask import VTKSegmentationMask
from VTKSegmentationActors import VTKSegmentationActors
from HelpingWindow import HelpingWindow
from TimeText import TimeText
from VTKPipeline import VTKPipeline

In [7]:
class MainWindow(QMainWindow):

    def __init__(self, switch = True, parent = None):
        QMainWindow.__init__(self, parent)
        
        init_time_step = 0
        self.switch = switch
              
        # Set up frame
        self.frame = QFrame()
        self.layout = QVBoxLayout()
                              
         # VTK widget
        self.vtkWidget = QVTKRenderWindowInteractor(self.frame)
        self.layout.addWidget(self.vtkWidget)
        
        self.path_dir = "../../Data/Test/Segmentation/"
        self.nr_time_steps = len([x for x in os.listdir(self.path_dir) if 'png' in x])-1
        self.vtkPipeline = VTKPipeline(self.nr_time_steps)
        self.changeReader(nr_time_step=0)
        self.vtkPipeline.SetWindow(self.vtkWidget)
        self.renderer = self.vtkPipeline.renderer
        self.vtkWidget.GetRenderWindow().AddRenderer(self.renderer)
        self.interactor = self.vtkWidget.GetRenderWindow().GetInteractor() 
        self.interactor.SetInteractorStyle(self.vtkPipeline.interactorStyle)
        
        self.frame.setLayout(self.layout)
        self.setCentralWidget(self.frame)
        
        # Toolbar with Slider
        self.toolbar = QToolBar("Time slider") 
        self.addToolBar(self.toolbar)
        self.createSliderToolbar()        
        
        # Menu Bar
        bar = self.menuBar()        
        new_folder = QAction("New Patient",self)
        new_folder.triggered.connect(self.openFolder)
        bar.addAction(new_folder)
              
        helping = QAction("Help", self)
        helping.triggered.connect(self.helping_button)
        bar.addAction(helping)
        
        quit = QAction("Quit",self) 
        quit.triggered.connect(self.closeWindow)
        bar.addAction(quit)
        
        # Show and initialize
        self.show()
        self.vtkWidget.Initialize()
        
    def closeWindow(self, s):
        logging.info("MainWindow: Close window.")
        self.close()
        
    def createSliderToolbar(self, init_value=0):
        logging.info("MainWindow: Create slider toolbar.")
        self.removeToolBar(self.toolbar)
        self.toolbar = QToolBar("Time slider") 
        
        self.slider = QSlider(Qt.Horizontal)
        self.slider.valueChanged.connect(self.timeStepChange)
        self.slider.setMinimum(0)
        self.slider.setMaximum(len(self.time_steps)-1)
        if init_value > len(self.time_steps)-1:
            init_value = len(self.time_steps)-1
        self.slider.setValue(init_value)
        self.slider.setTickPosition(QSlider.TicksBelow)
        self.slider.setTickInterval(1)    
        logging.info('MainWindow: Create slider {0}/{1}'.format(init_value, len(self.time_steps)-1))
        self.toolbar.addWidget(self.slider)
        
        self.toolbar.addSeparator()
        
        background_toggle = QCheckBox()
        background_toggle.setText("Background Toggle")
        background_toggle.setChecked(True)
        background_toggle.stateChanged.connect(self.backgroundToogleChange)
        
        self.toolbar.addWidget(background_toggle)
        self.addToolBar(self.toolbar)
        
    def backgroundToogleChange(self, s):
        pass
        if s == 0:
            logging.info('MainWindow: no background')
            self.vtkPipeline.RemoveDicomActor()
        elif s == 2:
            logging.info('MainWindow: background')
            self.vtkPipeline.AddDicomActor()
        
    def openFolder(self, s):        
        file_dialog = QFileDialog()
        file_dialog.setFileMode(QFileDialog.DirectoryOnly)
        file_dialog.setViewMode(QFileDialog.Detail)
        
        if file_dialog.exec_():
            self.path_dir = file_dialog.selectedFiles()[0]
            logging.info('MainWindow: Open folder {0}'.format(self.path_dir))
            self.changeReader()
            self.createSliderToolbar(init_value=self.slider.value())
            
    def changeReader(self, nr_time_step=None):
        # png files
        self.time_steps = [x for x in os.listdir(self.path_dir) if 'png' in x]
        self.nr_time_steps = len(self.time_steps)-1
        if nr_time_step is None:
            nr_time_step = min(self.nr_time_steps,self.slider.value())
            self.slider.setValue(nr_time_step)
            self.vtkPipeline.SetTimeText(self.slider.value(), self.nr_time_steps)
        path_dicom_dir = os.path.join(self.path_dir, self.time_steps[nr_time_step]) 
        path_dicom_files = [os.path.join(path_dicom_dir,x) for x in os.listdir(path_dicom_dir)]
        path_dicom_files = sorted(path_dicom_files, key=lambda x: int(x.split('slice')[-1].split('.')[0]))
        self.vtkPipeline.UpdateReader(path_dicom_files)
        # segmentation files
        path_data_mask = [os.path.join(self.path_dir,x) for x in os.listdir(self.path_dir) if 't0' in x or 't1' in x]
        #self.vtkPipeline.UpdateMask(path_data_mask)
        logging.info('MainWindow: Change png folder {0}'.format(path_dicom_dir))
        logging.info('MainWindow: Change mask folder {0}'.format(path_data_mask))
        
    def helping_button(self):
        logging.info("MainWindow: Open help window.")
        self.helpingWindow = HelpingWindow()
        self.helpingWindow.show()
            
    def timeStepChange(self,s):
        logging.info('MainWindow: Change step {0}/{1}'.format(self.slider.value(),self.nr_time_steps))
        self.vtkPipeline.SetTimeText(self.slider.value(), self.nr_time_steps)
        self.changeReader()       

In [8]:
app = QApplication(sys.argv)
window = MainWindow(True)
window.show()

app.exec_()

INFO:root:VTKPipeline: Update png reader with new first file ../../Data/Test/Segmentation/png/test_slice0.png
INFO:root:VTKPipeline: first mask folders ['../../Data/Test/Segmentation/t0', '../../Data/Test/Segmentation/t1 10-10-2020', '../../Data/Test/Segmentation/t1 18-10-2020']
INFO:root:VTKPipeline: Update png reader with new first file ../../Data/Test/Segmentation/png\test_slice0.png
INFO:root:MainWindow: Change png folder ../../Data/Test/Segmentation/png
INFO:root:MainWindow: Change mask folder ['../../Data/Test/Segmentation/t0', '../../Data/Test/Segmentation/t1 10-10-2020', '../../Data/Test/Segmentation/t1 18-10-2020']
INFO:root:MainWindow: Create slider toolbar.
INFO:root:MainWindow: Create slider 0/2
INFO:root:VTKPipeline: Setting new slice 11
INFO:root:VTKPipeline: Setting new slice 12
INFO:root:VTKPipeline: Setting new slice 13
INFO:root:VTKPipeline: Setting new slice 14
INFO:root:VTKPipeline: Setting new slice 15
INFO:root:VTKPipeline: Setting new slice 16
INFO:root:VTKPipeli

INFO:root:VTKPipeline: Setting new slice 0
INFO:root:VTKPipeline: Setting new slice 0
INFO:root:VTKPipeline: Setting new slice 0
INFO:root:VTKPipeline: Setting new slice 0
INFO:root:VTKPipeline: Setting new slice 0
INFO:root:VTKPipeline: Setting new slice 0
INFO:root:VTKPipeline: Setting new slice 0
INFO:root:VTKPipeline: Setting new slice 0
INFO:root:VTKPipeline: Setting new slice 0
INFO:root:VTKPipeline: Setting new slice 0
INFO:root:VTKPipeline: Setting new slice 0
INFO:root:VTKPipeline: Setting new slice 1
INFO:root:VTKPipeline: Setting new slice 2
INFO:root:VTKPipeline: Setting new slice 3
INFO:root:VTKPipeline: Setting new slice 4
INFO:root:VTKPipeline: Setting new slice 5
INFO:root:VTKPipeline: Setting new slice 6
INFO:root:VTKPipeline: Setting new slice 7
INFO:root:VTKPipeline: Setting new slice 8
INFO:root:VTKPipeline: Setting new slice 9
INFO:root:VTKPipeline: Setting new slice 10
INFO:root:VTKPipeline: Setting new slice 11
INFO:root:VTKPipeline: Setting new slice 12
INFO:roo

0